# Trabajo Práctico N° 3
## Programación paralela distribuida

### Objetivo
El objetvio de la práctica es desarrollar una solución distribuida implementando el paradigma de la programación paralela. Para ello, se utilizará mpi4py como librería del leguaje Python la cual provee de las clases y métodos necesarios para la implementación del estandar de comunicación MPI.

## Enunciado
Construir un programa que permita contabilidar la ventas diarias de diferentes categorías de productos generadas por 16 sucursales de una muy conocida cadena de supermercados.

Se cuenta con un dataset segmentado por sucursal con la siguiente estructura:
```
data = [
  {"sucursal_1": {"categoria_1": 300, "categoria_2": 1000}},
  {"sucursal_2": {"categoria_1": 300, "categoria_4": 1400}},
]
```

Por otro lado, disponemos de un dataset mas pequeño con el porcentaje de aumento para determinados productos:
```
descuentos = {
  "categoria_1": 0.10,
  "categoria_4": 0.25
}
```

**NOTA:** Ambos datasets **solo estarán disponibles desde nodo maestro (raiz)**

El programa en cuestión debe poder:
* Registrar el número de ventas totales por categoria.
* Implementar paralelismo con 2 o mas procesos.
* Registrar los tiempos de ejecución de cada proceso involucrado.
* PLUS: Informar cual fue la sucursal que registro el mayor descuento.



---



Debemos instalar la libreria de mpi4py para poder utilizarlo en el entorno de colab

In [ ]:
! pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4441921 sha256=0c6d0e92842d2d10fea5047864c9a6427f1315d419b81c7eaeae2c09ddd8226b
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [ ]:
%%writefile trabajo_practico_3.py
from mpi4py import MPI
import numpy as np
import time
import os


def generate_dataset(num_branches, num_categories):
    total = []
    for i in range(num_branches):
        branch_name = f"branch_{i + 1}"
        categories = {}
        for j in range(num_categories):
            category_name = f"category_{j + 1}"
            categories[category_name] = np.random.randint(1000)
        total.append({branch_name: categories})
    return total


def separate_dataset(dataset, num_processes):
    aux_dataset = []
    for data in dataset:
        for _, value in data.items():
            aux_dataset.append(value)
    return np.array_split(aux_dataset, num_processes)


def sum_partially_total_sales(sales_by_category):
  #
  # COMPLETAR CON CODIGO DEL ALUMNO
  #
  pass

def sum_total_sales(total_sales_a, total_sales_b, datatype=None):
  #
  # COMPLETAR CON CODIGO DEL ALUMNO
  #
  pass

def apply_discounts(partial_sales, discounts):
  #
  # COMPLETAR CON CODIGO DEL ALUMNO
  #
  pass

OP_SUM = MPI.Op.Create(sum_total_sales, commute=True)

NUM_BRANCHES = int(os.getenv("SUC", 4))
NUM_CATEGORIES = int(os.getenv("CAT", 4))


def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    print(f"Process {rank} of {size}")

    global_start_time = 0.0
    total_sales = 0
    if rank == 0:
        global_start_time = time.time()
        discounts = {
          "categoria_1": 0.10,
          "categoria_4": 0.25,
          "categoria_3": 0.15,
          "categoria_2": 0.20,
          "categoria_5": 0.12,
          "categoria_6": 0.18,
          "categoria_7": 0.14,
          "categoria_8": 0.22,
          "categoria_9": 0.17,
          "categoria_10": 0.11,
        }
      #
      # COMPLETAR CON CODIGO DEL ALUMNO (DE SER NECESARIO)
      #
    else:
        splitted_dataset = None
        discounts = None

    #
    # COMPLETAR CON CODIGO DEL ALUMNO (DE SER NECESARIO)
    #
    if rank == 0:
        global_end_time = time.time()
        total_execution_time = round(global_end_time - global_start_time, 10)
        print("TOTAL SALES", total_sales)
        print(f"Total execution time: {total_execution_time} seconds")


if __name__ == "__main__":
    main()

Overwriting trabajo_practico_3.py


In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO_PROC =   4#@param {type: "number"}
COUNT_SUC =   8#@param {type: "number"}
COUNT_CAT =   30000#@param {type: "number"}
# --------------------------------------------

! mpirun --oversubscribe --allow-run-as-root -x CAT=$COUNT_CAT -x SUC=$COUNT_SUC -np $NRO_PROC python trabajo_practico_3.py

Process 2 of 4
Process 3 of 4
Process 0 of 4
TOTAL SALES 0
Total execution time: 3.0994e-06 seconds
Process 1 of 4


## Programación paralela con memoria compartida

# Objetivo

En esta ocación disponemos únicamente de la cantidad de ventas por sucursal en un periodo dado.

El objetivo es calcular el total de ventas generadas en todo el periodo, implementando las directivas adecuadas de OpenMP para realizarlo de forma paralela.



In [ ]:
%%writefile trabajo_practico_3.cpp
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>

#define SEED 4
#define LIMIT_INF 0
#define LIMIT_SUP 100

int main(int argc, char* argv[]){
    if(argv[1] && argv[2])
    {
        int count_suc = atoi( argv[1] );
        int days = atoi( argv[2] );
        int ventas_x_sucursal[count_suc][days];
        int total = 0;
        int index = 0;
        int total_test = 0;

        srand(SEED);
        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < days; j++)
            {
                ventas_x_sucursal[i][j] = (rand() % (LIMIT_SUP - LIMIT_INF + 1)) + LIMIT_INF;
            }
        }

        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < days; j++)
            {
                printf("%d \t", ventas_x_sucursal[i][j]);
                total_test += ventas_x_sucursal[i][j];
            }
            printf("\n");

        }
        printf("Total Test: %d\n", total_test);
        printf("\n");

        // COMPLETAR CON CODIGO DEL ALUMNO
        printf("%d\n", total);
    }
    else
    {
        printf("Por favor, ingrese la cantidad de sucursales");
    }
}

Overwriting trabajo_practico_3.cpp


In [ ]:
!g++ -o trabajo_practico_3 -fopenmp trabajo_practico_3.cpp

In [ ]:
%env OMP_NUM_THREADS=4
!./trabajo_practico_3 5 3

env: OMP_NUM_THREADS=4
78 	30 	7 	
32 	71 	10 	
85 	11 	85 	
26 	11 	96 	
70 	80 	21 	
Total Test: 713

0
